# NBA Salary Data Cleaning

In [1]:
# Dependencies and Setup
import pandas as pd 
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Pull CSV
salary_df = pd.read_csv("Resources/salaries_1985to2018.csv")
salary_df.head()

,index,league,player_id,salary,season,season_end,season_start,team
0,0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [3]:
# Drop unnecessary columns
total_salary = salary_df[['player_id', 'salary']]
total_salary.head()

,player_id,salary
0,abdelal01,395000
1,abdelal01,494000
2,abdelal01,500000
3,abdelal01,805000
4,abdelal01,650000


In [4]:
# Print unique players
print(len(total_salary["player_id"].unique()))

2408


In [5]:
player_salary = total_salary.groupby(['player_id']).sum()
player_salary.head()

,salary
player_id,
abdelal01,2844000
abdulka01,8560000
abdulma02,19849500
abdulta01,37982800
abdursh01,95866000


In [6]:
# Print total rows
player_salary.count()

salary    2408
dtype: int64

In [7]:
# Rename player id and salary for merging
player_salary.index.rename("_id", inplace=True)
player_salary.rename(columns={"salary": "Total Earnings"}, inplace=True)
player_salary

,Total Earnings
_id,
abdelal01,2844000
abdulka01,8560000
abdulma02,19849500
abdulta01,37982800
abdursh01,95866000
...,...
zidekge01,2081200
zimmest01,950000
zipsepa01,2062611


In [ ]:
# Push clean csv files
filepath = Path('Resources/cleaned_salary_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
player_salary.to_csv(filepath) 

# Adjusting for Inflation
All inflation rate is as of 09/01/2023

In [8]:
cpi_file = pd.read_csv('Resources/CPIAUCNS.csv')
cpi_file.head()

,DATE,CPIAUCNS
0,1913-01-01,9.8
1,1913-02-01,9.8
2,1913-03-01,9.8
3,1913-04-01,9.8
4,1913-05-01,9.7


In [9]:
cpi_file['CPI_rate'] = cpi_file['CPIAUCNS'].iloc[-1]/cpi_file['CPIAUCNS']

In [10]:
cpi_file['DATE'] = pd.to_datetime(cpi_file['DATE'])

In [11]:
cpi_file.head()

,DATE,CPIAUCNS,CPI_rate
0,1913-01-01,9.8,31.407041
1,1913-02-01,9.8,31.407041
2,1913-03-01,9.8,31.407041
3,1913-04-01,9.8,31.407041
4,1913-05-01,9.7,31.730825


# Setting the Date for Merge

In [12]:
cleaned_df = salary_df[['player_id', 'salary', 'season_start']]
cleaned_df.head()

,player_id,salary,season_start
0,abdelal01,395000,1990
1,abdelal01,494000,1991
2,abdelal01,500000,1992
3,abdelal01,805000,1993
4,abdelal01,650000,1994


In [13]:
cleaned_df['month'] = 1
cleaned_df['day'] = 1

In [14]:
date_df = cleaned_df[['season_start', 'month', 'day']]
date_df.head()

,season_start,month,day
0,1990,1,1
1,1991,1,1
2,1992,1,1
3,1993,1,1
4,1994,1,1


In [15]:
date_df = date_df.rename(columns={'season_start':'year'})

In [16]:
date_df.head()

,year,month,day
0,1990,1,1
1,1991,1,1
2,1992,1,1
3,1993,1,1
4,1994,1,1


In [17]:
cleaned_df['DATE'] = pd.to_datetime(date_df)

In [18]:
cleaned_df.head()

,player_id,salary,season_start,month,day,DATE
0,abdelal01,395000,1990,1,1,1990-01-01
1,abdelal01,494000,1991,1,1,1991-01-01
2,abdelal01,500000,1992,1,1,1992-01-01
3,abdelal01,805000,1993,1,1,1993-01-01
4,abdelal01,650000,1994,1,1,1994-01-01


# Reorganizing DataFrame for Merge

In [19]:
just_date = cleaned_df[['player_id', 'salary', 'DATE']]
just_date

,player_id,salary,DATE
0,abdelal01,395000,1990-01-01
1,abdelal01,494000,1991-01-01
2,abdelal01,500000,1992-01-01
3,abdelal01,805000,1993-01-01
4,abdelal01,650000,1994-01-01
...,...,...,...
14158,zipsepa01,750000,2016-01-01
14159,zipsepa01,1312611,2017-01-01
14160,zizican01,1645200,2017-01-01
14161,zubaciv01,1034956,2016-01-01


# Merge DataFrames

In [20]:
merged1 = pd.merge(just_date, cpi_file, how='left', on="DATE")

In [21]:
merged1.head()

,player_id,salary,DATE,CPIAUCNS,CPI_rate
0,abdelal01,395000,1990-01-01,127.4,2.415926
1,abdelal01,494000,1991-01-01,134.6,2.286694
2,abdelal01,500000,1992-01-01,138.1,2.228740
3,abdelal01,805000,1993-01-01,142.6,2.158408
4,abdelal01,650000,1994-01-01,146.2,2.105260


In [22]:
merged1['salary'] = merged1['salary'].astype(float)

In [23]:
merged1['inflation_salary'] = merged1['CPI_rate'] * merged1['salary']

In [24]:
merged1.head()

,player_id,salary,DATE,CPIAUCNS,CPI_rate,inflation_salary
0,abdelal01,395000.0,1990-01-01,127.4,2.415926,9.542909e+05
1,abdelal01,494000.0,1991-01-01,134.6,2.286694,1.129627e+06
2,abdelal01,500000.0,1992-01-01,138.1,2.228740,1.114370e+06
3,abdelal01,805000.0,1993-01-01,142.6,2.158408,1.737519e+06
4,abdelal01,650000.0,1994-01-01,146.2,2.105260,1.368419e+06


# Finding Inflation Salary

In [25]:
completed_inflation = merged1[['player_id', 'inflation_salary']]
completed_inflation

,player_id,inflation_salary
0,abdelal01,9.542909e+05
1,abdelal01,1.129627e+06
2,abdelal01,1.114370e+06
3,abdelal01,1.737519e+06
4,abdelal01,1.368419e+06
...,...,...
14158,zipsepa01,9.743612e+05
14159,zipsepa01,1.663683e+06
14160,zizican01,2.085227e+06
14161,zubaciv01,1.344561e+06


In [26]:
completed_inflation['Total_inflation_salary'] = completed_inflation.groupby(['player_id'])['inflation_salary'].transform(sum)
completed_inflation = completed_inflation.drop_duplicates(subset=['player_id'])[['player_id','Total_inflation_salary']]


In [27]:
completed_inflation.head()

,player_id,Total_inflation_salary
0,abdelal01,6.304225e+06
5,abdulka01,2.406022e+07
9,abdulma02,4.177226e+07
18,abdulta01,6.558494e+07
28,abdursh01,1.630366e+08


# Turning Scientific Notation Easier to Read

In [28]:
completed_inflation['Total_inflation_salary'] = completed_inflation['Total_inflation_salary'].map("${:,.2f}".format)


In [29]:
completed_inflation.head()

,player_id,Total_inflation_salary
0,abdelal01,"$6,304,225.16"
5,abdulka01,"$24,060,224.72"
9,abdulma02,"$41,772,255.61"
18,abdulta01,"$65,584,943.25"
28,abdursh01,"$163,036,582.98"
